In [1]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil
import logging
logger = logging.getLogger(str(os.getpid()))

from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, resample_or_rolling_stats, timelag_predictors, timelag_predictors_target, roll_predictors_target
from wattile.time_processing import add_processed_time_columns
PROJECT_DIRECTORY = Path().resolve().parent.parent

# reading configs

In [2]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# main configs file
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)
##################################################################################
# code testing configs file
# with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["data_input"]["exp_dir"] = str(PROJECT_DIRECTORY / exp_dir)
configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / "data" / "Synthetic Site")

configs

{'data_input': {'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\data\\Synthetic Site',
  'data_config': 'Synthetic Site Config.json',
  'start_time': '2018-01-01T00:00:00-07:00',
  'end_time': '2022-01-01T00:00:00-07:00',
  'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
   'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'Synthetic Weather Station Direct Normal Irradiance',
   'Synthetic Weather Station Dry Bulb Temperature',
   'Synthetic Weather Station Global Horizontal Irradiance',
   'Synthetic Weather Station Relative Humidity',
   'Synthetic Weather Station Wind Speed'],
  'target_var': 'Synthetic Site Electricity Main Total Power',
  'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir'},
 'data_output': {'exp_dir': 'exp_dir',
  'plot_comparison': True,
  'plot_comparison_portion_start': 0.0,
  'plot_comparison_portion_end': 1.0},
 'data_processing': {'fea

In [3]:
configs["data_processing"]["feat_stats"]["active"] = True
configs["data_processing"]["feat_stats"]["window_width"] = "10min"

configs["data_processing"]["resample"]["bin_interval"] = "10min"
configs["data_processing"]["resample"]["bin_closed"] = "right"
configs["data_processing"]["resample"]["bin_label"] = "right"

configs["data_processing"]["feat_timelag"]["lag_count"] = 5
configs["data_processing"]["feat_timelag"]["lag_interval"] = "30min"

configs["data_processing"]["input_output_window"]["window_width_target"] = "40min"
configs["data_processing"]["input_output_window"]["window_width_futurecast"] = "30min"

configs["learning_algorithm"]["arch_version"] = "bravo"

# reading data

In [4]:
configs["target_feat_name"] = [configs["data_input"]["target_var"]]
data = read_dataset_from_file(configs)
data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


# data processing 1

In [5]:
# assert we have the correct columns and order them
data = correct_predictor_columns(configs, data)

# sort and trim data specified time period
data = correct_timestamps(configs, data)

# Add time-based features
data = add_processed_time_columns(data, configs)

# Add statistics features
data = resample_or_rolling_stats(data, configs)

data

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,HOD_binary_reg_0_min,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-0.258819,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,104.800003
2021-12-01 07:10:00+00:00,14.782966,-1.496226,-0.207964,55.112000,-1.873579,17.809999,9.701875,0.953717,-0.300706,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,101.250000
2021-12-01 07:20:00+00:00,15.474850,-1.405313,-0.540707,54.608002,-1.918852,18.610001,8.612455,0.939693,-0.342020,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,96.900002
2021-12-01 07:30:00+00:00,16.103886,-1.378900,-1.741706,54.661999,-1.879938,19.760000,10.149275,0.923880,-0.382683,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:40:00+00:00,16.996296,-1.642929,-2.147240,54.734001,-2.029242,19.320000,6.907860,0.906308,-0.422618,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,101.099998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:20:00+00:00,10.851442,-1.571301,-0.566701,41.988201,-1.790767,23.760000,0.000000,0.996195,-0.087156,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,107.963600
2021-12-08 06:30:00+00:00,10.889771,-1.550848,-0.655083,42.024200,-1.834018,23.770000,0.000000,0.991445,-0.130526,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,105.972198
2021-12-08 06:40:00+00:00,10.827111,-1.430384,-0.478316,40.960400,-1.865315,24.330000,0.000000,0.984808,-0.173648,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,113.689598


# data processing 2

In [6]:
# reading configuration parameters
lag_interval = configs["data_processing"]["feat_timelag"]["lag_interval"]
lag_count = configs["data_processing"]["feat_timelag"]["lag_count"]
window_width_target = configs["data_processing"]["input_output_window"][
    "window_width_target"
]
window_width_futurecast = configs["data_processing"]["input_output_window"][
    "window_width_futurecast"
]
bin_interval = configs["data_processing"]["resample"]["bin_interval"]
initial_num = (pd.Timedelta(window_width_target) // pd.Timedelta(bin_interval)) + 1
target_var = configs["data_input"]["target_var"]
target_temp = data[target_var].copy()

# shift target for futurecast
data[target_var] = target_temp.shift(freq="-" + window_width_futurecast)

data

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,HOD_binary_reg_0_min,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-0.258819,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:10:00+00:00,14.782966,-1.496226,-0.207964,55.112000,-1.873579,17.809999,9.701875,0.953717,-0.300706,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,101.099998
2021-12-01 07:20:00+00:00,15.474850,-1.405313,-0.540707,54.608002,-1.918852,18.610001,8.612455,0.939693,-0.342020,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,100.050003
2021-12-01 07:30:00+00:00,16.103886,-1.378900,-1.741706,54.661999,-1.879938,19.760000,10.149275,0.923880,-0.382683,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,102.949997
2021-12-01 07:40:00+00:00,16.996296,-1.642929,-2.147240,54.734001,-2.029242,19.320000,6.907860,0.906308,-0.422618,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,97.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:20:00+00:00,10.851442,-1.571301,-0.566701,41.988201,-1.790767,23.760000,0.000000,0.996195,-0.087156,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,105.309204
2021-12-08 06:30:00+00:00,10.889771,-1.550848,-0.655083,42.024200,-1.834018,23.770000,0.000000,0.991445,-0.130526,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,104.239998
2021-12-08 06:40:00+00:00,10.827111,-1.430384,-0.478316,40.960400,-1.865315,24.330000,0.000000,0.984808,-0.173648,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,NaN


In [7]:
initial_num

5

In [8]:
# split predictors and target
target = data[target_var]
data = data.drop(target_var, axis=1)
data_orig = data

# Pad the exogenous variables
temp_holder = list()
temp_holder.append(data_orig)
for i in range(1, lag_count + 1):
    shifted = (
        data_orig.shift(freq=i * lag_interval)
        .astype("float32")
        .add_suffix("_lag{}".format(i))
    )
    temp_holder.append(shifted)
temp_holder.reverse()
data = pd.concat(temp_holder, axis=1)

# Do fine padding for future predictions. Create a new df to preserve memory usage.
local = pd.DataFrame()
for i in range(0, initial_num):
    if i == 0:
        local["{}_lag_{}".format(target_var, i)] = target.shift(i)
    else:
        local["{}_lag_{}".format(target_var, i)] = target.shift(
            freq="-" + (i * bin_interval)
        )

data = pd.concat([data, local], axis=1)
    
data

,Synthetic Weather Station Dew Point Temperature_min_lag5,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag5,Synthetic Weather Station Direct Normal Irradiance_min_lag5,Synthetic Weather Station Dry Bulb Temperature_min_lag5,Synthetic Weather Station Global Horizontal Irradiance_min_lag5,Synthetic Weather Station Relative Humidity_min_lag5,Synthetic Weather Station Wind Speed_min_lag5,sin_HOD_min_lag5,cos_HOD_min_lag5,HOD_binary_reg_0_min_lag5,...,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power_lag_0,Synthetic Site Electricity Main Total Power_lag_1,Synthetic Site Electricity Main Total Power_lag_2,Synthetic Site Electricity Main Total Power_lag_3,Synthetic Site Electricity Main Total Power_lag_4
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,-0.493776,0.869589,98.050003,101.099998,100.050003,102.949997,97.900002
2021-12-01 07:10:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,-0.493776,0.869589,101.099998,100.050003,102.949997,97.900002,104.800003
2021-12-01 07:20:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,-0.493776,0.869589,100.050003,102.949997,97.900002,104.800003,99.949997
2021-12-01 07:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,-0.493776,0.869589,102.949997,97.900002,104.800003,99.949997,99.408005
2021-12-01 07:40:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,-0.493776,0.869589,97.900002,104.800003,99.949997,99.408005,100.849998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 08:50:00+00:00,10.851442,-1.571301,-0.566701,41.988201,-1.790767,23.760000,0.0,0.996195,-0.087156,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-08 09:00:00+00:00,10.889771,-1.550848,-0.655083,42.024200,-1.834018,23.770000,0.0,0.991445,-0.130526,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-08 09:10:00+00:00,10.827111,-1.430384,-0.478316,40.960400,-1.865315,24.330000,0.0,0.984808,-0.173648,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data_temp = data.loc[:, 
         (data.columns.str.contains("Synthetic Weather Station Dew Point Temperature_min"))|
         (data.columns.str.contains("Synthetic Site Electricity Main Total Power"))
        ]

data_temp

,Synthetic Weather Station Dew Point Temperature_min_lag5,Synthetic Weather Station Dew Point Temperature_min_lag4,Synthetic Weather Station Dew Point Temperature_min_lag3,Synthetic Weather Station Dew Point Temperature_min_lag2,Synthetic Weather Station Dew Point Temperature_min_lag1,Synthetic Weather Station Dew Point Temperature_min,Synthetic Site Electricity Main Total Power_lag_0,Synthetic Site Electricity Main Total Power_lag_1,Synthetic Site Electricity Main Total Power_lag_2,Synthetic Site Electricity Main Total Power_lag_3,Synthetic Site Electricity Main Total Power_lag_4
Timestamp,,,,,,,,,,,
2021-12-01 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,15.413733,98.050003,101.099998,100.050003,102.949997,97.900002
2021-12-01 07:10:00+00:00,NaN,NaN,NaN,NaN,NaN,14.782966,101.099998,100.050003,102.949997,97.900002,104.800003
2021-12-01 07:20:00+00:00,NaN,NaN,NaN,NaN,NaN,15.474850,100.050003,102.949997,97.900002,104.800003,99.949997
2021-12-01 07:30:00+00:00,NaN,NaN,NaN,NaN,15.413733,16.103886,102.949997,97.900002,104.800003,99.949997,99.408005
2021-12-01 07:40:00+00:00,NaN,NaN,NaN,NaN,14.782966,16.996296,97.900002,104.800003,99.949997,99.408005,100.849998
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 08:50:00+00:00,10.851442,10.798893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-08 09:00:00+00:00,10.889771,10.975215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-08 09:10:00+00:00,10.827111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data_temp.to_csv("./output1.csv")